### Training an Agent to solve the openAI gym's mountain car problem.

1. [Environment Details](https://github.com/openai/gym/wiki/MountainCar-v0)
2. [Leaderboard Page](https://github.com/openai/gym/wiki/Leaderboard#mountaincar-v0)

#### Mountain Car Problem: Get an under powered car to the top of a hill (top = 0.5 position)
1. Action Space: motor = (left, neutral, right):int
2. Observation Space: np.array([position, velocity])
    - Veclocity = (-0.07, 0.07)
    - Position = (-1.2, 0.6)

In [3]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import sleep
import time
from scipy.special import softmax
# np.random.seed(0)

env = gym.make('MountainCar-v0')

ModuleNotFoundError: No module named 'gym'

In [ ]:
print(env.action_space)
print(env.observation_space)

### Agent Class

In [ ]:
class State:
    def __init__(self):
        self.pos = None
        self.vel = None

class Agent:
    def __init__(self, env):
        self.velocityLimit = np.array([env.observation_space.low[1], env.observation_space.high[1]])
        self.positionLimit = np.array([env.observation_space.low[0], env.observation_space.high[0]])
        self.velocityStep, self.positionStep = 0.005, 0.1
        self.velocitySpace = np.arange(self.velocityLimit[0], self.velocityLimit[1] 
                                       + self.velocityStep, self.velocityStep)
        self.positionSpace = np.arange(self.positionLimit[0], self.positionLimit[1] 
                                       + self.positionStep, self.positionStep)
        self.m, self.n, self.n_action = len(self.velocitySpace), len(self.positionSpace), 3
        self.Q = np.full(shape = (self.m, self.n, 3),
                                       fill_value = 0.0, dtype = np.float32)
        self.collectiveRecord = []
        self.success = []
        
    def getActionValueIndex(self, state):
        posOffset = state[0] - self.positionLimit[0]
        velOffset = state[1] - self.velocityLimit[0]
        posInd = posOffset // self.positionStep
        velInd = velOffset // self.velocityStep
        
        return np.array([velInd, posInd], dtype= np.int)
  
    def getAction(self, state):
        ind = self.getActionValueIndex(state, 0)
        p = self.Policy[ind[0], ind[1], :]
        action = np.random.choice([0, 1, 2], size = 1, p = p)
        return action[0]

## Training: SARSA($\lambda$) Agent

### Method: 
1. Sarsa($\lambda$) with $\lambda$ = 0.8.
2. States are represeted by discretiziing the position and velocity. 
2. epsilon-greedy policy with starting epsilon = 0.8 and decay rate = 0.995. 
3. starting learning-rate(alpha) = 0.8 and decay rate = 0.999.
4. No reward shaping.

#### Goal: 
- Solving the problem means average reward of -110 or more for 100 consecutive iterations.

In [ ]:
def runSarsa(eps, alpha, LAMBDA, epsDecay, alphaDecay, numEpisodes, agent, env):
    for i_eps in tqdm(range(1, numEpisodes + 1)):
        state = env.reset()
        agent.e = np.zeros(shape = (agent.m, agent.n, 3))
        gamma = 1.0
        ind = agent.getActionValueIndex(state)
        if np.random.random() < 1 - eps:
            action = np.argmax(agent.Q[ind[0], ind[1], :]) 
        else:
            action = np.random.randint(0, 3)

        for t in range(201):
            ind = agent.getActionValueIndex(state)
            nextState, reward, done, info = env.step(action)
            nextInd = agent.getActionValueIndex(nextState)

            if np.random.random() < 1 - eps:
                nextAction = np.argmax(agent.Q[nextInd[0], nextInd[1], :]) 
            else: 
                nextAction = np.random.randint(0, 3)

            delta = reward + gamma * agent.Q[nextInd[0],nextInd[1],nextAction] - agent.Q[ind[0],ind[1],action]
            agent.e[ind[0],ind[1],action] += 1

            agent.Q = np.add(agent.Q, np.multiply(alpha * delta, agent.e))
            agent.e = np.multiply(gamma * LAMBDA, agent.e)

            if done: 
                if t < 199:
                    agent.success.append((i_eps, t))
                agent.collectiveRecord.append(-t)
                eps = max(0.0, eps * epsDecay)
                alpha = max(0.0, alpha * alphaDecay)
                break
            state = nextState
            action = nextAction
# env.close()

In [ ]:
env = gym.make('MountainCar-v0')
env.seed(0)
agent = Agent(env)
print("Q Shape = ",agent.Q.shape)
sleep(0.5)
numEpisodes = 20000
runSarsa(0.8, 0.2, 0.8, 0.998, 0.999, numEpisodes, agent, env)

1. Print **reward** over 100 previous episodes for all the episodes during training

In [ ]:
fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(agent.collectiveRecord[:],'.')
plt.yticks(range(-110, -200, -10))
plt.ylabel("reward")
plt.xlabel("episode_number")
plt.grid()
plt.show()

2. Print **average reward** over 100 previous episodes for all the episodes during training

In [ ]:
avgReward = []
for i in range(100, numEpisodes):
    avgReward.append(np.mean(agent.collectiveRecord[i - 100:i]))
fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(avgReward, '.')
plt.yticks(range(-110, -200, -10))
# plt.xticks(changed_eps)
plt.ylabel("Avg reward for last 100 episodes")
plt.xlabel("episode_number")
plt.grid()
plt.show()

## Test Over 100 episodes

In [ ]:
runSarsa(0.0, 0.0, 1.0, 1.0, 1.0, 100, agent, env)

In [ ]:
print("Average Reward on 100 training episodes = {}".format(np.mean(agent.collectiveRecord[-100:])))
fig, ax = plt.subplots(figsize = (14, 8))
plt.plot(agent.collectiveRecord[-100:], '-')
plt.yticks(range(-110, -200, -10))
plt.title("Test Results")
plt.ylabel("Reward")
plt.xlabel("episode_number")
plt.grid()
plt.show()

---

## SARSA($\lambda$) with replacing traces

In [ ]:
def runSarsaWithRT(eps, alpha, LAMBDA, epsDecay, alphaDecay, numEpisodes, agent, env):
    for i_eps in tqdm(range(1, numEpisodes + 1)):
        state = env.reset()
        agent.e = np.zeros(shape = (agent.m, agent.n, 3))
        gamma = 1.0
        ind = agent.getActionValueIndex(state)
        if np.random.random() < 1 - eps:
            action = np.argmax(agent.Q[ind[0], ind[1], :]) 
        else:
            action = np.random.randint(0, 3)

        for t in range(201):
            ind = agent.getActionValueIndex(state)
            nextState, reward, done, info = env.step(action)
            nextInd = agent.getActionValueIndex(nextState)

            if np.random.random() < 1 - eps:
                nextAction = np.argmax(agent.Q[nextInd[0], nextInd[1], :]) 
            else: 
                nextAction = np.random.randint(0, 3)

            delta = reward + gamma * agent.Q[nextInd[0],nextInd[1],nextAction] - agent.Q[ind[0],ind[1],action]
#             agent.e[ind[0], ind[1], :] = 0
            agent.e[ind[0], ind[1], action] = 1

            agent.Q = np.add(agent.Q, np.multiply(alpha * delta, agent.e))
            agent.e = np.multiply(gamma * LAMBDA, agent.e)

            if done: 
                if t < 199:
                    agent.success.append((i_eps, t))
                agent.collectiveRecord.append(-t)
                eps = max(0.0, eps * epsDecay)
                alpha = max(0.0, alpha * alphaDecay)
                break
            state = nextState
            action = nextAction
# env.close()

In [ ]:
env = gym.make('MountainCar-v0')
env.seed(0)
np.random.seed(0)
agent = Agent(env)
print("Q Shape = ",agent.Q.shape)
sleep(0.5)
numEpisodes = 20000
runSarsaWithRT(0.8, 0.2, 0.8, 0.995, 0.999, numEpisodes, agent, env)

2. Print **average reward** over 100 previous episodes for all the episodes during training

In [ ]:
avgReward = []
for i in range(100, numEpisodes):
    avgReward.append(np.mean(agent.collectiveRecord[i - 100:i]))
fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(avgReward, '.')
plt.yticks(range(-110, -200, -10))
# plt.xticks(changed_eps)
plt.ylabel("Avg reward for last 100 episodes")
plt.xlabel("episode_number")
plt.grid()
plt.show()

In [ ]:
runSarsaWithRT(0.0, 0.0, 1.0, 1.0, 1.0, 100, agent, env)

In [ ]:
print("Average Reward on 100 training episodes = {}".format(np.mean(agent.collectiveRecord[-100:])))
fig, ax = plt.subplots(figsize = (14, 8))
plt.plot(agent.collectiveRecord[-100:], '-')
plt.yticks(range(-110, -200, -10))
plt.title("Test Results")
plt.ylabel("Reward")
plt.xlabel("episode_number")
plt.grid()
plt.show()